# 01. SHAP Explanations | تفسيرات SHAP

## 📚 Learning Objectives

By completing this notebook, you will:
- Understand the key concepts of this topic
- Apply the topic using Python code examples
- Practice with small, realistic datasets or scenarios

## 🔗 Prerequisites

- ✅ Basic Python
- ✅ Basic NumPy/Pandas (when applicable)

---

## Official Structure Reference

This notebook supports **Course 06, Unit 4** requirements from `DETAILED_UNIT_DESCRIPTIONS.md`.

---


# 01. SHAP Explanations | تفسيرات SHAP

## 🚨 THE PROBLEM: AI Systems Are "Black Boxes" | المشكلة: أنظمة الذكاء الاصطناعي "صناديق سوداء"

**Remember the transition from Unit 3?**

We completed Unit 3: Privacy and Security, where we learned:
- How to protect data and ensure privacy
- How to comply with regulations
- How to build secure AI systems

**But there's a new challenge:**

**AI systems also raise transparency and accountability concerns!**

**The Problem**: As we build AI systems, we need to consider:
- ❌ **Transparency**: How do we explain AI decisions?
- ❌ **Accountability**: Who is responsible for AI outcomes?
- ❌ **Explainability**: How do we make AI understandable?
- ❌ **Auditability**: How do we track and verify AI behavior?

**We've learned:**
- ✅ How to build private and secure AI systems (Unit 3)
- ✅ How to ensure GDPR compliance
- ✅ How to protect data

**But we haven't learned:**
- ❌ How to **explain** AI model decisions
- ❌ How to **understand** why models make certain predictions
- ❌ How to **interpret** model behavior
- ❌ How to **provide explanations** to users

**We need explainability techniques** to:
1. Explain individual predictions (local explanations)
2. Understand overall model behavior (global explanations)
3. Identify important features
4. Make AI decisions transparent

**This notebook solves that problem** by teaching you SHAP (SHapley Additive exPlanations) for model interpretability!

---

## 📚 Prerequisites (What You Need First) | المتطلبات الأساسية

**BEFORE starting this notebook**, you should have completed:
- ✅ **Unit 1: Ethics Foundations** - Understanding ethical principles
- ✅ **Unit 2: Bias and Justice** - Understanding fairness
- ✅ **Unit 3: Privacy and Security** - Understanding privacy and security
- ✅ **Basic Python knowledge**: Functions, data manipulation, ML concepts
- ✅ **Basic ML knowledge**: Model training, predictions, feature importance

**If you haven't completed these**, you might struggle with:
- Understanding why explainability matters
- Knowing how to interpret model predictions
- Understanding feature importance concepts

---

## 🔗 Where This Notebook Fits | مكان هذا الدفتر

**This is the FIRST example in Unit 4** - it teaches you how to explain AI decisions!

**Why this example FIRST?**
- **Before** you can use other explanation methods, you need to understand SHAP
- **Before** you can ensure accountability, you need explainability
- **Before** you can build transparent systems, you need explanation techniques

**Builds on**: 
- 📓 Unit 1: Ethics Foundations (ethical principles)
- 📓 Unit 2: Bias and Justice (fairness concepts)
- 📓 Unit 3: Privacy and Security (privacy and security)

**Leads to**: 
- 📓 Example 2: LIME Explanations (alternative explanation method)
- 📓 Example 3: Counterfactual Analysis (what-if explanations)
- 📓 Example 4: Accountability Frameworks (accountability structures)
- 📓 Example 5: Human-in-the-Loop (HITL approaches)
- 📓 Example 6: Transparency Tools (transparency frameworks)

**Why this order?**
1. SHAP provides **foundation** for explainability (needed before other methods)
2. SHAP teaches **feature importance** (critical for understanding)
3. SHAP shows **local and global explanations** (essential for transparency)

---

## The Story: Opening the Black Box | القصة: فتح الصندوق الأسود

Imagine you're a doctor using AI to diagnose patients. **Before** explainability, you'd get a diagnosis but wouldn't know why (black box!). **After** using SHAP, you can see which symptoms most influenced the diagnosis - transparent and understandable!

Same with AI: **Before** we have models that make predictions but we can't explain why, now we learn SHAP - calculate feature contributions to understand each prediction! **After** SHAP, we can explain why models make specific decisions!

---

## Why SHAP Explanations Matter | لماذا تهم تفسيرات SHAP؟

SHAP explanations are essential for ethical AI:
- **Transparency**: Understand why models make decisions
- **Trust**: Build user confidence through explanations
- **Debugging**: Identify model errors and biases
- **Compliance**: Meet explainability requirements
- **Accountability**: Enable responsibility for AI outcomes

## Learning Objectives | أهداف التعلم
1. Understand SHAP values and their meaning
2. Learn how to calculate SHAP values
3. Generate local explanations for individual predictions
4. Generate global explanations for overall model behavior
5. Visualize SHAP explanations
6. Interpret SHAP results

In [1]:
"""
Unit 4: Interpretability, Transparency, and Accountability
Example 1: SHAP Explanations
This example demonstrates SHAP (SHapley Additive exPlanations) for model interpretability:
- SHAP values calculation
- Global and local explanations
- Feature importance visualization
"""
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')
# Try to import SHAP, use simplified version if not available
try:
    import shap
    SHAP_AVAILABLE = True
except ImportError:
    SHAP_AVAILABLE = False
    print("Note: SHAP library not available. Using simplified SHAP implementation.")
plt.rcParams['font.size'] = 10
plt.rcParams['figure.figsize'] = (14, 8)
sns.set_style("whitegrid")
# ============================================================================
# SIMPLIFIED SHAP IMPLEMENTATION (if SHAP not available)
# ============================================================================
def calculate_shap_values_simple(model, X, feature_names):
    """
    Simplified SHAP value calculation using permutation importance
    """
    baseline_pred = model.predict_proba(X)[:, 1].mean()
    shap_values = []
    for i in range(len(X)):
        sample = X[i:i+1]
        base_pred = model.predict_proba(sample)[0, 1]
        sample_shap = []
        for j in range(X.shape[1]):
            # Permute feature j
            X_permuted = X.copy()
            X_permuted[:, j] = sample[0, j]
            perm_pred = model.predict_proba(X_permuted)[:, 1].mean()
            # SHAP value approximation
            shap_val = base_pred - perm_pred
            sample_shap.append(shap_val)
        shap_values.append(sample_shap)
    return np.array(shap_values)
# ============================================================================
# GENERATE DATASET
# ============================================================================
def generate_dataset(n_samples=1000):
    """
    Generate synthetic dataset for SHAP demonstration
    """
    np.random.seed(42)
    # Features
    age = np.random.randint(18, 80, n_samples)
    income = np.random.normal(50000, 20000, n_samples)
    credit_score = np.random.normal(650, 100, n_samples)
    debt_ratio = np.random.uniform(0.1, 0.6, n_samples)
    # Target (loan approval)
    approval_prob = (credit_score / 850 * 0.4 +
                     (income / 100000) * 0.3 +
                     (1 - debt_ratio) * 0.2 +
                     (age / 80) * 0.1 +
                     np.random.normal(0, 0.05, n_samples))
    approval = (approval_prob > 0.5).astype(int)
    df = pd.DataFrame({
        'age': age, 'income': income,
        'credit_score': credit_score,
        'debt_ratio': debt_ratio,
        'approved': approval
    })
    return df
# ============================================================================
# SHAP EXPLANATIONS
# ============================================================================
def explain_with_shap(model, X, feature_names, use_library=True):
    """
    Generate SHAP explanations for the model
    """
    if SHAP_AVAILABLE and use_library:
        # Use actual SHAP library
        explainer = shap.TreeExplainer(model)
        shap_values = explainer.shap_values(X)
        if isinstance(shap_values, list):
            shap_values = shap_values[1]  # For binary classification, use positive class
        return shap_values, explainer
    else:
        # Use simplified implementation
        shap_values = calculate_shap_values_simple(model, X, feature_names)
        return shap_values, None
# ============================================================================
# VISUALIZATIONS
# ============================================================================
def plot_shap_summary(shap_values, X, feature_names):
    """
    Plot SHAP summary plot
    """
    fig, ax = plt.subplots(figsize=(12, 8))
    # Calculate mean absolute SHAP values for feature importance
    mean_shap = np.abs(shap_values).mean(axis=0)
    # Sort by importance
    indices = np.argsort(mean_shap)
    y_pos = np.arange(len(feature_names))
    colors = plt.cm.RdYlGn(mean_shap / mean_shap.max())
    ax.barh(y_pos, mean_shap[indices], color=colors[indices])
    ax.set_yticks(y_pos)
    ax.set_yticklabels([feature_names[i] for i in indices])
    ax.set_xlabel('Mean |SHAP Value|', fontsize=11, fontweight='bold')
    ax.set_title('SHAP Feature Importance Summary', fontsize=12, fontweight='bold')
    ax.grid(axis='x', alpha=0.3)
    plt.tight_layout()
    plt.savefig('unit4-transparency-accountability', dpi=300, bbox_inches='tight')
    print("✅ Saved: shap_summary.png")
    plt.close()
def plot_shap_waterfall(shap_values, X, feature_names, sample_idx=0):
    """
    Plot SHAP waterfall plot for a single prediction
    """
    sample_shap = shap_values[sample_idx]
    sample_values = X[sample_idx]
    # Sort by absolute SHAP value
    indices = np.argsort(np.abs(sample_shap))[::-1]
    fig, ax = plt.subplots(figsize=(12, 8))
    # Calculate cumulative SHAP values
    cumulative = np.cumsum([0] + list(sample_shap[indices]))
    # Plot waterfall
    for i in range(len(indices)):
        idx = indices[i]
        color = 'red' if sample_shap[idx] < 0 else 'green'
        ax.barh(i, sample_shap[idx], left=cumulative[i], color=color, alpha=0.7)
        ax.text(cumulative[i] + sample_shap[idx]/2, i, 
               f'{feature_names[idx]}\n={sample_values[idx]:.2f}',
               ha='center', va='center', fontsize=9)
    ax.set_yticks(range(len(indices)))
    ax.set_yticklabels([feature_names[i] for i in indices])
    ax.set_xlabel('SHAP Value', fontsize=11, fontweight='bold')
    ax.set_title(f'SHAP Waterfall Plot (Sample {sample_idx})', fontsize=12, fontweight='bold')
    ax.axvline(x=0, color='black', linestyle='--', linewidth=1)
    ax.grid(axis='x', alpha=0.3)
    plt.tight_layout()
    plt.savefig('unit4-transparency-accountability', dpi=300, bbox_inches='tight')
    print("✅ Saved: shap_waterfall.png")
    plt.close()
def plot_shap_dependence(shap_values, X, feature_names, feature_idx=0):
    """
    Plot SHAP dependence plot for a specific feature
    """
    fig, ax = plt.subplots(figsize=(10, 6))
    feature_values = X[:, feature_idx]
    feature_shap = shap_values[:, feature_idx]
    scatter = ax.scatter(feature_values, feature_shap, alpha=0.5, c=feature_shap, 
                        cmap='RdBu_r', s=30)
    ax.set_xlabel(feature_names[feature_idx], fontsize=11, fontweight='bold')
    ax.set_ylabel('SHAP Value', fontsize=11, fontweight='bold')
    ax.set_title(f'SHAP Dependence Plot: {feature_names[feature_idx]}', fontsize=12, fontweight='bold')
    ax.axhline(y=0, color='black', linestyle='--', linewidth=1)
    ax.grid(alpha=0.3)
    plt.colorbar(scatter, ax=ax, label='SHAP Value')
    plt.tight_layout()
    plt.savefig('unit4-transparency-accountability', dpi=300, bbox_inches='tight')
    print("✅ Saved: shap_dependence.png")
    plt.close()
# ============================================================================
# MAIN EXECUTION
# ============================================================================
if __name__ == "__main__":
    print("="*80)
    print("Unit 4 - Example 1: SHAP Explanations")
    print("="*80)
    # Generate dataset
    print("\nGenerating dataset...")
    df = generate_dataset(n_samples=1000)
    print(f"Dataset shape: {df.shape}")
    # Prepare data
    feature_names = ['age', 'income', 'credit_score', 'debt_ratio']
    X = df[feature_names].values
    y = df['approved'].values
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42, stratify=y
    )
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    # Train model
    print("\nTraining Random Forest model...")
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train_scaled, y_train)
    train_acc = accuracy_score(y_train, model.predict(X_train_scaled))
    test_acc = accuracy_score(y_test, model.predict(X_test_scaled))
    print(f"Training Accuracy: {train_acc:.4f}")
    print(f"Test Accuracy: {test_acc:.4f}")
    # Calculate SHAP values
    print("\nCalculating SHAP values...")
    shap_values, explainer = explain_with_shap(
        model, X_test_scaled[:100], feature_names, use_library=SHAP_AVAILABLE
    )
    print(f"SHAP values shape: {shap_values.shape}")
    # Create visualizations
    print("\n" + "="*80)
    print("Creating Visualizations...")
    print("="*80)
    plot_shap_summary(shap_values, X_test_scaled[:100], feature_names)
    plot_shap_waterfall(shap_values, X_test_scaled[:100], feature_names, sample_idx=0)
    plot_shap_dependence(shap_values, X_test_scaled[:100], feature_names, feature_idx=2)
    # Summary
    print("\n" + "="*80)
    print("SUMMARY")
    print("="*80)
    print("\nKey Takeaways:")
    print("1. SHAP values explain individual predictions")
    print("2. SHAP summary shows global feature importance")
    print("3. Waterfall plots show how features contribute to a specific prediction")
    print("4. Dependence plots show how SHAP values vary with feature values")
    print("5. SHAP provides model-agnostic explanations")
    print("="*80 + "\n")


Note: SHAP library not available. Using simplified SHAP implementation.
Unit 4 - Example 1: SHAP Explanations

Generating dataset...
Dataset shape: (1000, 5)

Training Random Forest model...
Training Accuracy: 1.0000
Test Accuracy: 0.9367

Calculating SHAP values...


SHAP values shape: (100, 4)

Creating Visualizations...
✅ Saved: shap_summary.png


✅ Saved: shap_waterfall.png
✅ Saved: shap_dependence.png

SUMMARY

Key Takeaways:
1. SHAP values explain individual predictions
2. SHAP summary shows global feature importance
3. Waterfall plots show how features contribute to a specific prediction
4. Dependence plots show how SHAP values vary with feature values
5. SHAP provides model-agnostic explanations



---

## 🚫 When SHAP Explanations Hit a Limitation | عندما تصل تفسيرات SHAP إلى حد

### The Limitation We Discovered

We've learned SHAP for explaining model predictions. **But there's still a challenge:**

**What if we need a simpler, faster explanation method that works with any model?**

SHAP works well when:
- ✅ We need mathematically rigorous explanations
- ✅ We can afford computational cost
- ✅ We want feature importance based on game theory

**But sometimes we need:**
- ❌ **Faster explanations** (SHAP can be slow for large datasets)
- ❌ **Simpler methods** (easier to understand and implement)
- ❌ **Model-agnostic approaches** (work with any black-box model)
- ❌ **Local explanations** (explain individual predictions quickly)

### Why This Is a Problem

When SHAP is too slow or complex:
- We may need faster explanations for real-time systems
- We may want simpler methods for non-technical users
- We may need alternatives for different use cases
- We may want to compare different explanation methods

### The Solution: LIME Explanations

We need **LIME (Local Interpretable Model-agnostic Explanations)** to:
1. Provide fast local explanations
2. Work with any black-box model
3. Offer simpler, more intuitive explanations
4. Enable real-time explainability

**This is exactly what we'll learn in the next notebook: LIME Explanations!**

---

## ➡️ Next Steps | الخطوات التالية

**You've completed this notebook!** Now you understand:
- ✅ How to use SHAP for model explanations
- ✅ How to generate local and global explanations
- ✅ **The limitation**: We need faster, simpler alternatives!

**Next notebook**: `02_lime_explanations.ipynb`
- Learn about LIME for fast local explanations
- Understand model-agnostic explainability
- Compare SHAP vs LIME approaches
